In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings

In [2]:
myData=pd.read_csv('imputedData.csv')
myData.head()

#One hot encoding of categorical variables

#Create list with features to be dummified cols.
nonum_feats_names = ['Month','OperatingSystems','Browser','Region','VisitorType','Weekend']


dataModel = pd.concat([myData[['Administrative', 'Administrative_Duration', 'Informational','Informational_Duration', 
                              'ProductRelated','ProductRelated_Duration','BounceRates','ExitRates','PageValues',
                               'TrafficType','SpecialDay']],
                       pd.get_dummies(myData[nonum_feats_names].astype('category')),myData['Revenue']],axis=1)
                      
dataModel.info()


#Label Encoding of revenue

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
myData['Revenue'] = le.fit_transform(myData['Revenue'])
myData['Revenue'].value_counts()

myData['Revenue'].head()

# getting dependent and independent variables

x = dataModel
# removing the target column revenue from x
x = x.drop(['Revenue'], axis = 1)

y = myData['Revenue']

# checking the shapes
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 57 columns):
Administrative                   12330 non-null int64
Administrative_Duration          12330 non-null float64
Informational                    12330 non-null int64
Informational_Duration           12330 non-null float64
ProductRelated                   12330 non-null int64
ProductRelated_Duration          12330 non-null float64
BounceRates                      12330 non-null float64
ExitRates                        12330 non-null float64
PageValues                       12330 non-null float64
TrafficType                      12330 non-null int64
SpecialDay                       12330 non-null float64
Month_Aug                        12330 non-null uint8
Month_Dec                        12330 non-null uint8
Month_Feb                        12330 non-null uint8
Month_Jul                        12330 non-null uint8
Month_June                       12330 non-null uint8
Month_Mar    

In [3]:
# splitting the data

from sklearn.model_selection import train_test_split

X_Train, X_Test, y_Train, y_Test = train_test_split(x, y, test_size = 0.3, random_state = 42,shuffle=True,stratify=y)

# checking the shapes

print("Shape of x_train :", X_Train.shape)
print("Shape of y_train :", y_Train.shape)
print("Shape of x_test :", X_Test.shape)
print("Shape of y_test :", y_Test.shape)

Shape of x_train : (8631, 56)
Shape of y_train : (8631,)
Shape of x_test : (3699, 56)
Shape of y_test : (3699,)


In [4]:
pd.DataFrame(y_Train,columns=['Revenue']).Revenue.value_counts(normalize=True)

0    0.845209
1    0.154791
Name: Revenue, dtype: float64

In [5]:
#SCALING WITH STANDARD Z SCORE SCALER
#Scaling the data first fitting it and transforming the training set
#to later apply the fit to transform the test set.
from sklearn.preprocessing import StandardScaler

num_cols_names = ['Administrative', 'Administrative_Duration', 'Informational','Informational_Duration', 
                              'ProductRelated','ProductRelated_Duration','BounceRates','ExitRates','PageValues',
                               'TrafficType','SpecialDay']
#Instantiate Satandard Scaler
scaler = StandardScaler()
#Fit transform the numerical features in the training dataset to a new dataframe
scaled_numfeats_train = pd.DataFrame(scaler.fit_transform(X_Train[num_cols_names]), 
                                     columns=num_cols_names, index= X_Train.index)
#Integrate scaled values to the training set
for col in num_cols_names:
    X_Train[col] = scaled_numfeats_train[col]
    
    
#Transform the numerical features inthe training dataset to a new dataframe
scaled_numfeats_test = pd.DataFrame(scaler.transform(X_Test[num_cols_names]),
                                    columns=num_cols_names, index= X_Test.index)
#Integrate scaled values to the test set
for col in num_cols_names:
    X_Test[col] = scaled_numfeats_test[col]

C:\Users\neera\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\neera\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
X_Train.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,TrafficType,...,Region_5,Region_6,Region_7,Region_8,Region_9,VisitorType_New_Visitor,VisitorType_Other,VisitorType_Returning_Visitor,Weekend_False,Weekend_True
7460,0.508002,-0.200488,-0.3947,-0.244336,-0.399509,-0.483009,-0.454346,-0.668356,-0.320111,0.223142,...,0,0,0,0,0,1,0,0,1,0
4687,-0.397089,-0.229562,-0.3947,-0.244336,-0.283724,-0.388159,-0.454346,-0.668356,-0.320111,-0.023995,...,0,0,0,0,0,0,0,1,1,0
790,-0.698787,-0.456792,-0.3947,-0.244336,-0.422666,-0.538272,-0.454346,-0.251598,-0.320111,-0.271132,...,0,0,0,0,0,0,0,1,1,0
9057,1.111396,-0.102539,-0.3947,-0.244336,0.341513,-0.040404,-0.308034,-0.544755,-0.320111,0.964551,...,0,0,0,0,0,0,0,1,1,0
225,-0.698787,-0.456792,-0.3947,-0.244336,-0.677393,-0.669486,-0.454346,1.173716,-0.320111,1.705961,...,0,0,0,0,0,0,0,1,0,1


In [7]:
X_Train.drop(labels=['Browser_1','BounceRates','ProductRelated','VisitorType_Returning_Visitor'], 
                 axis=1, inplace=True)
X_Test.drop(labels=['Browser_1','BounceRates','ProductRelated','VisitorType_Returning_Visitor'], 
                axis=1, inplace=True)

C:\Users\neera\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
X_Train = X_Train.drop(['Weekend_True','Region_2','Region_1','Month_Jul','Month_Feb','Informational','Browser_9','Browser_12',
                        'Browser_11','Region_8','Region_3','OperatingSystems_6','OperatingSystems_1','Month_June',
                        'Browser_8','Browser_7','Browser_3','Browser_13','Browser_10','VisitorType_Other','Region_8','Region_7',
                        'Region_5','OperatingSystems_5','Month_Aug'],axis=1)
X_Test = X_Test.drop(['Weekend_True','Region_2','Region_1','Month_Jul','Month_Feb','Informational','Browser_9','Browser_12',
                        'Browser_11','Region_8','Region_3','OperatingSystems_6','OperatingSystems_1','Month_June',
                        'Browser_8','Browser_7','Browser_3','Browser_13','Browser_10','VisitorType_Other','Region_8','Region_7',
                        'Region_5','OperatingSystems_5','Month_Aug'],axis=1)

In [9]:
X_Train = X_Train.drop(['OperatingSystems_4','OperatingSystems_7','OperatingSystems_8','Browser_5','Region_9'],axis=1)
X_Test = X_Test.drop(['OperatingSystems_4','OperatingSystems_7','OperatingSystems_8','Browser_5','Region_9'],axis=1)

print(X_Train.shape, X_Test.shape)

(8631, 23) (3699, 23)


In [10]:
from imblearn.over_sampling import SMOTE
sm=SMOTE(random_state=33,sampling_strategy=0.6)
sm
X_Train_res, y_Train_res = sm.fit_sample(X_Train, y_Train)

In [11]:
pd.DataFrame(y_Train_res,columns=['Revenue']).Revenue.value_counts(normalize=True)

0    0.625
1    0.375
Name: Revenue, dtype: float64

In [12]:
print("Shape of x_train :", X_Train_res.shape) 
print("Shape of y_train :", y_Train_res.shape)

Shape of x_train : (11672, 23)
Shape of y_train : (11672,)


# Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression
classifier_lg = LogisticRegression() 
classifier_lg.fit(X_Train_res,y_Train_res)
print(classifier_lg)

y_pred=classifier_lg.predict(X_Test)
warnings.filterwarnings("default")

# evaluating the model
from sklearn.metrics import roc_auc_score
print("Training Accuracy :", classifier_lg.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_lg.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

from sklearn.metrics import classification_report,confusion_matrix
print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, cross_validate


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_lg, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
Training Accuracy : 0.8313913639479096
Testing Accuracy : 0.8845633955123007
ROC AUC Score : 0.786021142273141
*******************************************************************************************
CONFUSION MATRIX
[[2904  223]
 [ 204  368]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      3127
           1       0.62      0.64      0.63       572

    accuracy                           0.88      3699
   macro avg       0.78      0.79      0.78      3699
weighted avg       0.89      0.88      0.8

C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CROSS VALIDATION METRICS
Mean Accuracy : 0.8302775505280204
Standard Deviation : 0.005752434663923313
Mean precision score : 0.8538870761134303
Standard Deviation precision score : 0.006761760988823408
Mean recall score : 0.6604939239104313
Standard Deviation recall score : 0.01803087811207932
Mean f1 score : 0.7446835687707559
Standard Deviation f1 score : 0.011411195415976865
Mean AUC ROC Score : 0.796319081766554
Standard Deviation AUC ROC Score : 0.00806033427111586
*******************************************************************************************


C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# Logistic Regression Hyper-parameter Tuning

In [14]:
from sklearn.model_selection import GridSearchCV

base_classifier = LogisticRegression(penalty='l2')
params = [
    {
     'C' : [0.1, 0.01,1],
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'C': [0.1, 0.01, 1],
                          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                     'saga']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
********************************************************

# Logistic Regression Hyper parameter tuned model

In [15]:
from sklearn.linear_model import LogisticRegression
classifier_lg = LogisticRegression(C=1,solver='liblinear') 
classifier_lg.fit(X_Train_res,y_Train_res)
print(classifier_lg)

y_pred=classifier_lg.predict(X_Test)

# evaluating the model
from sklearn.metrics import roc_auc_score
print("Training Accuracy :", classifier_lg.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_lg.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

from sklearn.metrics import classification_report,confusion_matrix
print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, cross_validate


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_lg, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)
Training Accuracy : 0.8313913639479096
Testing Accuracy : 0.8845633955123007
ROC AUC Score : 0.786021142273141
*******************************************************************************************
CONFUSION MATRIX
[[2904  223]
 [ 204  368]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      3127
           1       0.62      0.64      0.63       572

    accuracy                           0.88      3699
   macro avg       0.78      0.79      0.78      3699
weighted avg       0.89      0.88      

# Naive Bayes

In [16]:
from sklearn.naive_bayes import GaussianNB
classifier_nb = GaussianNB() 
classifier_nb.fit(X_Train_res,y_Train_res)
print(classifier_nb)

y_pred=classifier_nb.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_nb.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_nb.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_nb, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

GaussianNB(priors=None, var_smoothing=1e-09)
Training Accuracy : 0.7485435229609322
Testing Accuracy : 0.7280346039470127
ROC AUC Score : 0.7484359660167142
*******************************************************************************************
CONFUSION MATRIX
[[2248  879]
 [ 127  445]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.95      0.72      0.82      3127
           1       0.34      0.78      0.47       572

    accuracy                           0.73      3699
   macro avg       0.64      0.75      0.64      3699
weighted avg       0.85      0.73      0.76      3699

*******************************************************************************************
CROSS VALIDATION METRICS
Mean Accuracy : 0.7475148840956775
Standard Deviation : 0.00918783014454218
Mean precision score : 0.6301505763438531
Standard Deviation precision sc

# KNN

In [17]:
from sklearn.neighbors import KNeighborsClassifier
classifier_knn = KNeighborsClassifier() 
classifier_knn.fit(X_Train_res,y_Train_res)
print(classifier_knn)

y_pred=classifier_knn.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_knn.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_knn.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_knn, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')
Training Accuracy : 0.924006168608636
Testing Accuracy : 0.8104893214382265
ROC AUC Score : 0.738638040884603
*******************************************************************************************
CONFUSION MATRIX
[[2635  492]
 [ 209  363]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.93      0.84      0.88      3127
           1       0.42      0.63      0.51       572

    accuracy                           0.81      3699
   macro avg       0.68      0.74      0.70      3699
weighted avg       0.85      0.81      0.82      3699

*******************************************************************************************
CROSS VALIDATION METRICS
Mean Accu

# KNN Hyper-parameter tuning

In [18]:
base_classifier = KNeighborsClassifier()
params = [
    {
     'n_neighbors':[5,8,10,15],
        'leaf_size':[30,35,40],
        'weights':['uniform', 'distance'],
        'algorithm':['auto', 'ball_tree','kd_tree','brute']
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=-1,
             param_grid=[{'algorithm': ['auto', 'ball_tree', 'kd_tree',
                                        'brute'],
                          'leaf_size': [30, 35, 40],
                          'n_neighbors': [5, 8, 10, 15],
                          'weights': ['uniform', 'distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
*******************************************************************************************
Best Hyper Parameters: {'algorithm': 'auto', 'leaf_size': 30, 'n

# KNN Hyper-parameter tuned model

In [19]:
classifier_knn = KNeighborsClassifier(algorithm='auto',leaf_size=30,n_neighbors=5,weights='distance') 
classifier_knn.fit(X_Train_res,y_Train_res)
print(classifier_knn)

y_pred=classifier_knn.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_knn.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_knn.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_knn, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='distance')
Training Accuracy : 1.0
Testing Accuracy : 0.8123817247904839
ROC AUC Score : 0.7361861834999027
*******************************************************************************************
CONFUSION MATRIX
[[2647  480]
 [ 214  358]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.93      0.85      0.88      3127
           1       0.43      0.63      0.51       572

    accuracy                           0.81      3699
   macro avg       0.68      0.74      0.70      3699
weighted avg       0.85      0.81      0.83      3699

*******************************************************************************************
CROSS VALIDATION METRICS
Mean Accuracy : 0.896

# Support Vector Machine

In [20]:
from sklearn.svm import SVC
classifier_svc = SVC() 
classifier_svc.fit(X_Train_res,y_Train_res)
print(classifier_svc)

y_pred=classifier_svc.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_svc.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_svc.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_svc, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

C:\Users\neera\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
Training Accuracy : 0.8664324880054832
Testing Accuracy : 0.8845633955123007
ROC AUC Score : 0.8038768474889357
*******************************************************************************************
CONFUSION MATRIX
[[2879  248]
 [ 179  393]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.94      0.92      0.93      3127
           1       0.61      0.69      0.65       572

    accuracy                           0.88      3699
   macro avg       0.78      0.80      0.79      3699
weighted avg       0.89      0.88      0.89      3699

*********************************************************************

C:\Users\neera\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarni

CROSS VALIDATION METRICS
Mean Accuracy : 0.8614672619761311
Standard Deviation : 0.008319500978091435
Mean precision score : 0.852879657675056
Standard Deviation precision score : 0.011670334663992529
Mean recall score : 0.7621767342716529
Standard Deviation recall score : 0.02046745728479886
Mean f1 score : 0.8048172954511124
Standard Deviation f1 score : 0.013182109122609316
Mean AUC ROC Score : 0.8416090654089347
Standard Deviation AUC ROC Score : 0.010341020582847063
*******************************************************************************************


# Support Vector Machine hyper-parameter tuning

In [21]:
base_classifier = SVC()
params = [
    {
     'C': [0.1,1,5,7,8,9,10, 100, 1000],
        'gamma': [1,0.1,0.01,0.002,0.0005,0.001,0.0001],
        'kernel': ['rbf']
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'C': [0.1, 1, 5, 7, 8, 9, 10, 100, 1000],
                          'gamma': [1, 0.1, 0.01, 0.002, 0.0005, 0.001, 0.0001],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
*******************************************************************************************
Best Hyper Parameters: {'C': 9, 'gamma': 1, 'kernel': 'rbf'}
Best Accuracy Score: 0.9425119945167924
Best Estimators: SVC(C=9, cache_size=200, clas

# Support Vector Machine hyper-parameter tuned model

In [22]:
from sklearn.svm import SVC
classifier_svc = SVC(C=9,gamma=1,kernel='rbf') 
classifier_svc.fit(X_Train_res,y_Train_res)
print(classifier_svc)

y_pred=classifier_svc.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_svc.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_svc.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_svc, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

SVC(C=9, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)
Training Accuracy : 0.9924605894448252
Testing Accuracy : 0.8502297918356313
ROC AUC Score : 0.6357246047844065
*******************************************************************************************
CONFUSION MATRIX
[[2959  168]
 [ 386  186]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.88      0.95      0.91      3127
           1       0.53      0.33      0.40       572

    accuracy                           0.85      3699
   macro avg       0.71      0.64      0.66      3699
weighted avg       0.83      0.85      0.84      3699

***************************************************************************************

# AdaBoost

In [23]:
from sklearn. ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

classifier_adb = AdaBoostClassifier() 
classifier_adb.fit(X_Train_res,y_Train_res)
print(classifier_adb)

y_pred=classifier_adb.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_adb.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_adb.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_adb, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)
Training Accuracy : 0.8935058259081563
Testing Accuracy : 0.8756420654230873
ROC AUC Score : 0.7886010296067859
*******************************************************************************************
CONFUSION MATRIX
[[2860  267]
 [ 193  379]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.94      0.91      0.93      3127
           1       0.59      0.66      0.62       572

    accuracy                           0.88      3699
   macro avg       0.76      0.79      0.77      3699
weighted avg       0.88      0.88      0.88      3699

*******************************************************************************************
CROSS VALIDATION METRICS
Mean Accuracy : 0.8894083190988475
Standard Deviation : 0.0

# AdaBoost Hyper-parameter tuning

In [24]:
base_classifier = AdaBoostClassifier()
params = [
    {
     'n_estimators': [50,100,200,500],
          'learning_rate': [1,0.1,0.01,0.001],
          'base_estimator':[DecisionTreeClassifier(max_depth=1)]
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                          base_estimator=None,
                                          learning_rate=1.0, n_estimators=50,
                                          random_state=None),
             iid='warn', n_jobs=-1,
             param_grid=[{'base_estimator': [DecisionTreeClassifier(class_weight=None,
                                                                    criterion='gini',
                                                                    max_depth=1,
                                                                    max_features=None,
                                                                    max_leaf_nodes=None,
                                                                    min_impurity_decrease=0.0,
                                                                    min_impurity_split=None,
                           

# AdaBoost Hyper-parameter tuned model

In [25]:
classifier_adb = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best'),learning_rate=1, n_estimators=500) 
classifier_adb.fit(X_Train_res,y_Train_res)
print(classifier_adb)

y_pred=classifier_adb.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_adb.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_adb.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_adb, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                             

# Gradient Boosting

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
classifier_gb = GradientBoostingClassifier() 
classifier_gb.fit(X_Train_res,y_Train_res)
print(classifier_gb)

y_pred=classifier_gb.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_gb.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_gb.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_gb, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
Training Accuracy : 0.9169808087731323
Testing Accuracy : 0.8886185455528521
ROC AUC Score : 0.8084179970972424
*******************************************************************************************
CONFUSION MATRIX
[[2891  236]
 [ 176  396]]
****************************************************************************

# Gradient Boosting Hyper-parameter tuning

In [27]:
base_classifier = GradientBoostingClassifier()
params = [
    {
    'n_estimators': [100,150,200,500],
          'learning_rate': [1,0.1,0.01,0.001],
        'max_depth': [1, 3, 5]
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
                                                  presort

# Gradient Boosting Hyper-parameter tuned model

In [28]:
from sklearn.ensemble import GradientBoostingClassifier
classifier_gb = GradientBoostingClassifier(learning_rate=0.1, n_estimators=200,max_depth=5) 
classifier_gb.fit(X_Train_res,y_Train_res)
print(classifier_gb)

y_pred=classifier_gb.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_gb.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_gb.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_gb, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
Training Accuracy : 0.9655586017820424
Testing Accuracy : 0.8929440389294404
ROC AUC Score : 0.7895495134861941
*******************************************************************************************
CONFUSION MATRIX
[[2937  190]
 [ 206  366]]
****************************************************************************

# Bagging Tree

In [29]:
from sklearn import tree
classifier_bagt = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1)) 
classifier_bagt.fit(X_Train_res,y_Train_res)
print(classifier_bagt)

y_pred=classifier_bagt.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_bagt.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_bagt.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_bagt, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=1,
                                                        splitter='best'),
       

# Bagging Tree hyper-parameter tuning

In [30]:
base_classifier = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
params = [
    {
    'n_estimators': [10,20,50],
        'max_features': [10,15,23],
        'max_samples':[10,15,20]
        
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                                               criterion='gini',
                                                                               max_depth=None,
                                                                               max_features=None,
                                                                               max_leaf_nodes=None,
                                                                               min_impurity_decrease=0.0,
                                                                               min_impurity_split=None,
                                                                               min_samples_leaf=1,
                                                                               min_samples_split=2,
                                                  

# Bagging Tree hyper-parameter tuned model

In [31]:
classifier_bagt = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1),
                                    max_features=23, max_samples=20, n_estimators=50) 
classifier_bagt.fit(X_Train_res,y_Train_res)
print(classifier_bagt)

y_pred=classifier_bagt.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_bagt.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_bagt.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_bagt, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=1,
                                                        splitter='best'),
       

# Decision Tree

In [32]:
classifier_dt = DecisionTreeClassifier()
classifier_dt.fit(X_Train_res,y_Train_res)
print(classifier_dt)

y_pred=classifier_dt.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_dt.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_dt.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_dt, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
Training Accuracy : 1.0
Testing Accuracy : 0.8496891051635577
ROC AUC Score : 0.7375394432877644
*******************************************************************************************
CONFUSION MATRIX
[[2814  313]
 [ 243  329]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.92      0.90      0.91      3127
           1       0.51      0.58      0.54       572

    accuracy                           0.85      3699
   macro avg       0.72      0

# Decision Tree Hyper-parameter tuning

In [33]:
base_classifier = DecisionTreeClassifier()
params = [
    {
    'criterion':['gini','entropy'],
          'max_features': ['auto', 'sqrt', 'log2'],
          'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
          'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10,11],
          'random_state':[123],
          'max_depth':[1,2,3,4,5,6]
        
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_depth': [1, 2, 3, 4, 5, 6],
                  

# Decision Tree Hyper-parameter tuned model

In [34]:
classifier_dt = DecisionTreeClassifier(criterion='entropy', max_depth=6,
                                       max_features='auto', min_samples_leaf=1, min_samples_split=13, random_state=123)
classifier_dt.fit(X_Train_res,y_Train_res)
print(classifier_dt)

y_pred=classifier_dt.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_dt.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_dt.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_dt, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=13,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=123, splitter='best')
Training Accuracy : 0.7754455106237149
Testing Accuracy : 0.8148148148148148
ROC AUC Score : 0.6733447237124883
*******************************************************************************************
CONFUSION MATRIX
[[2746  381]
 [ 304  268]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      3127
           1       0.41      0.47      0.44       572

    accuracy                           0.81      3699
   macro avg 

# RANDOM FOREST

In [35]:
classifier_rf = RandomForestClassifier(n_estimators=200)
classifier_rf.fit(X_Train_res,y_Train_res)
print(classifier_rf)

y_pred=classifier_rf.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_rf.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_rf.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_rf, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Training Accuracy : 1.0
Testing Accuracy : 0.8861854555285212
ROC AUC Score : 0.7905516693092645
*******************************************************************************************
CONFUSION MATRIX
[[2905  222]
 [ 199  373]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.94      0.93      0.93      3127
           1       0.63      0.65      0.64       572

# RF Hyper-parameter tuning

In [36]:
## Hyper Parameter Tuning
print('Parameters currently in use:\n')
print(classifier_rf.get_params())
print("\n")

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 300, num = 5)] ## play with start and stop

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 20,num = 5)] ## change 10,20 and 2
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,15]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,10]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

from sklearn.model_selection import RandomizedSearchCV

rf_random = RandomizedSearchCV(estimator = classifier_rf, param_distributions = random_grid, n_iter = 100, cv = 3, 
                               verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_Train_res, y_Train_res)
print("*******************************************************************************************")
print("Best Hyper Parameters:",rf_random.best_params_)
print("Best Accuracy Score:",rf_random.best_score_)
print("Best Estimators:",rf_random.best_estimator_)
print("*******************************************************************************************")

Parameters currently in use:

{'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


{'n_estimators': [50, 112, 175, 237, 300], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 12, 15, 17, 20, None], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [1, 2, 4, 10], 'bootstrap': [True, False]}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   57.6s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.0min finished


*******************************************************************************************
Best Hyper Parameters: {'n_estimators': 112, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': True}
Best Accuracy Score: 0.9203221384509939
Best Estimators: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=112,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
*******************************************************************************************


# RF Hyper-parameter tuned model

In [37]:
classifier_rf = RandomForestClassifier(n_estimators=112,min_samples_split=2,min_samples_leaf=1,max_features='sqrt',max_depth=
                                       None,bootstrap=True)
classifier_rf.fit(X_Train_res,y_Train_res)
print(classifier_rf)

y_pred=classifier_rf.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_rf.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_rf.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_rf, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=112,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Training Accuracy : 1.0
Testing Accuracy : 0.8883482022168153
ROC AUC Score : 0.7939735352591125
*******************************************************************************************
CONFUSION MATRIX
[[2910  217]
 [ 196  376]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.94      0.93      0.93      3127
           1       0.63      0.66      0.65       572

# XGBOOST

In [38]:
from xgboost import XGBClassifier
classifier_xgb = XGBClassifier()
classifier_xgb.fit(X_Train_res,y_Train_res)
print(classifier_xgb)

y_pred=classifier_xgb.predict(X_Test.values)

# evaluating the model
print("Training Accuracy :", classifier_xgb.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_xgb.score(X_Test.values, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_xgb, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
Training Accuracy : 0.9119259766963673
Testing Accuracy : 0.8856447688564477
ROC AUC Score : 0.8109444920285982
*******************************************************************************************
CONFUSION MATRIX
[[2874  253]
 [ 170  402]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.94      0.92      0.93      3127
           1       0.61      0.70      0.66       572

    

# XGBOOST Hyper-parameter tuning

In [39]:
base_classifier = XGBClassifier()
params = [
    {
   'n_estimators': [100,150,200],
          'learning_rate': [1,0.1,0.01,0.015,0.025,0.05],
        'n_jobs':[-1]
        
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'learning_rate': [1, 0.1, 0.01, 0.015, 0.025, 0.05],
                          'n_estimators': [100, 150, 200], 'n_jobs': [-1]}],
            

# XGBOOST Hyper-parameter tuned model

In [40]:
classifier_xgb = XGBClassifier(learning_rate=0.1, n_estimators=200, n_jobs=-1)
classifier_xgb.fit(X_Train_res,y_Train_res)
print(classifier_xgb)

y_pred=classifier_xgb.predict(X_Test.values)

# evaluating the model
print("Training Accuracy :", classifier_xgb.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_xgb.score(X_Test.values, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_xgb, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
Training Accuracy : 0.9305174777244688
Testing Accuracy : 0.8905109489051095
ROC AUC Score : 0.7938242601658015
*******************************************************************************************
CONFUSION MATRIX
[[2920  207]
 [ 198  374]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      3127
           1       0.64      0.65      0.65       572

   

# VOTING CLASSIFIER

In [41]:
lr_vc = LogisticRegression(C=1,solver='liblinear')
svm_vc=SVC(C=9,gamma=1,kernel='rbf')
knn_vc=KNeighborsClassifier(algorithm='auto',leaf_size=30,n_neighbors=5,weights='distance')
nb_vc=GaussianNB()
dt_vc=DecisionTreeClassifier(criterion='entropy', max_depth=6,
                                       max_features='auto', min_samples_leaf=1, min_samples_split=13, random_state=123)
rf_vc=RandomForestClassifier(n_estimators=112,min_samples_split=2,min_samples_leaf=1,max_features='sqrt',max_depth=
                                       None,bootstrap=True)

evc = VotingClassifier(estimators= [('dt',dt_vc),('svm',svm_vc),('knn',knn_vc),('nb',nb_vc),('rf',rf_vc),('lr',lr_vc)], 
                       voting = 'hard')
evc.fit(X_Train_res,y_Train_res)
print(evc.fit)


y_pred=evc.predict(X_Test)

# evaluating the model
print("Training Accuracy :", evc.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", evc.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = evc, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

<bound method VotingClassifier.fit of VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(class_weight=None,
                                                     criterion='entropy',
                                                     max_depth=6,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=13,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort=False,
                                                     random_state=123,
                                      

# STACKING

In [42]:
## Algorithm 1: xgboost
pred_val_xgb=classifier_xgb.predict(X_Train_res)
test_pred_xgb=classifier_xgb.predict(X_Test.values)

## Algorithm 2: Random Forest
pred_val_rf=classifier_rf.predict(X_Train_res)
test_pred_rf=classifier_rf.predict(X_Test)

## Algorithm 3: Decision Tree
pred_val_dt=classifier_dt.predict(X_Train_res)
test_pred_dt=classifier_dt.predict(X_Test)

stacked_predictions=np.column_stack((pred_val_rf,pred_val_xgb,pred_val_dt)) ## Prediction by the algorithms on training data
#stacked_predictions[0:10]
stacked_test_predictions=np.column_stack((test_pred_rf,test_pred_xgb,test_pred_dt)) ## Prediction by the algorithms on testing data
#stacked_test_predictions[0:10]

## Building Meta Model
lr_stack = LogisticRegression()
lr_stack.fit(stacked_predictions,y_Train_res) ## stacked_predictions=X_train

y_pred_stack=lr_stack.predict(stacked_test_predictions) ## stacked_test_predictions=X_test
y_pred_stack


# evaluating the model
cm = confusion_matrix(y_Test, y_pred_stack)
print(cm)
print(classification_report(y_Test, y_pred_stack)) 

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = lr_stack, X = stacked_predictions, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

[[2910  217]
 [ 196  376]]
              precision    recall  f1-score   support

           0       0.94      0.93      0.93      3127
           1       0.63      0.66      0.65       572

    accuracy                           0.89      3699
   macro avg       0.79      0.79      0.79      3699
weighted avg       0.89      0.89      0.89      3699

CROSS VALIDATION METRICS
Mean Accuracy : 1.0
Standard Deviation : 0.0
Mean precision score : 1.0
Standard Deviation precision score : 0.0
Mean recall score : 1.0
Standard Deviation recall score : 0.0
Mean f1 score : 1.0
Standard Deviation f1 score : 0.0
Mean AUC ROC Score : 1.0
Standard Deviation AUC ROC Score : 0.0
*******************************************************************************************


C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

# BLENDING

In [43]:
## Algorithm 1: xgboost
pred_val_xgb=classifier_xgb.predict(X_Train_res)
test_pred_xgb=classifier_xgb.predict(X_Test.values)

## Algorithm 2: Random Forest
pred_val_rf=classifier_rf.predict(X_Train_res)
test_pred_rf=classifier_rf.predict(X_Test)

## Algorithm 3: Decision Tree
pred_val_dt=classifier_dt.predict(X_Train_res)
test_pred_dt=classifier_dt.predict(X_Test)

stacked_predictions=np.column_stack((pred_val_rf,pred_val_xgb,pred_val_dt)) ## Prediction by the algorithms on training data
#stacked_predictions[0:10]
stacked_test_predictions=np.column_stack((test_pred_rf,test_pred_xgb,test_pred_dt)) ## Prediction by the algorithms on testing data
#stacked_test_predictions[0:10]

stacked_predictions=pd.DataFrame(stacked_predictions)
stacked_test_predictions=pd.DataFrame(stacked_test_predictions)


## Building Meta Model
lr_stack = LogisticRegression()
lr_stack.fit(stacked_predictions,y_Train_res) ## stacked_predictions=X_train

y_pred_stack=lr_stack.predict(stacked_test_predictions) ## stacked_test_predictions=X_test
y_pred_stack


# evaluating the model
cm = confusion_matrix(y_Test, y_pred_stack)
print(cm)
print(classification_report(y_Test, y_pred_stack)) 

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = lr_stack, X = stacked_predictions, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

[[2910  217]
 [ 196  376]]
              precision    recall  f1-score   support

           0       0.94      0.93      0.93      3127
           1       0.63      0.66      0.65       572

    accuracy                           0.89      3699
   macro avg       0.79      0.79      0.79      3699
weighted avg       0.89      0.89      0.89      3699

CROSS VALIDATION METRICS
Mean Accuracy : 1.0
Standard Deviation : 0.0
Mean precision score : 1.0
Standard Deviation precision score : 0.0
Mean recall score : 1.0
Standard Deviation recall score : 0.0
Mean f1 score : 1.0
Standard Deviation f1 score : 0.0
Mean AUC ROC Score : 1.0
Standard Deviation AUC ROC Score : 0.0
*******************************************************************************************


C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
